<a href="https://colab.research.google.com/github/Naaastenka/Project1/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [ ]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [ ]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [ ]:
pd.read_sql(
    '''
    SELECT 'CRIM' AS column_name, SUM(CRIM IS NULL OR CRIM = '') AS missing_count FROM boston
    UNION ALL
    SELECT 'ZN' AS column_name, SUM(ZN IS NULL OR ZN = '') AS missing_count FROM boston
    UNION ALL
    SELECT 'INDUS' AS column_name, SUM(INDUS IS NULL OR INDUS = '') AS missing_count FROM boston
    UNION ALL
    SELECT 'CHAS' AS column_name, SUM(CHAS IS NULL OR CHAS = '') AS missing_count FROM boston
    UNION ALL
    SELECT 'NOX' AS column_name, SUM(NOX IS NULL OR NOX = '') AS missing_count FROM boston
    ''',
    con,
)

,column_name,missing_count
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [ ]:
pd.read_sql(
    '''
    SELECT 'CRIM' AS column_name, COUNT(DISTINCT CRIM) AS unique_count FROM boston
    UNION ALL
    SELECT 'ZN' AS column_name, COUNT(DISTINCT ZN) AS unique_count FROM boston
    UNION ALL
    SELECT 'INDUS' AS column_name, COUNT(DISTINCT INDUS) AS unique_count FROM boston
    UNION ALL
    SELECT 'CHAS' AS column_name, COUNT(DISTINCT CHAS) AS unique_count FROM boston
    UNION ALL
    SELECT 'NOX' AS column_name, COUNT(DISTINCT NOX) AS unique_count FROM boston
    ''',
    con,
)

,column_name,unique_count
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [ ]:
pd.read_sql(
    '''
    WITH MinValues AS (
        SELECT
            MIN(CRIM) AS CRIM_min,
            MIN(ZN) AS ZN_min,
            MIN(INDUS) AS INDUS_min,
            MIN(CHAS) AS CHAS_min,
            MIN(NOX) AS NOX_min
        FROM boston
    ),
    MedianValues AS (
        SELECT
            (SELECT CRIM FROM boston ORDER BY CRIM LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) AS CRIM_median,
            (SELECT ZN FROM boston ORDER BY ZN LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) AS ZN_median,
            (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) AS INDUS_median,
            (SELECT CHAS FROM boston ORDER BY CHAS LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) AS CHAS_median,
            (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) FROM boston)/2) AS NOX_median
    )
    SELECT 'CRIM' AS column_name FROM MinValues, MedianValues WHERE CRIM_min = CRIM_median
    UNION ALL
    SELECT 'ZN' AS column_name FROM MinValues, MedianValues WHERE ZN_min = ZN_median
    UNION ALL
    SELECT 'INDUS' AS column_name FROM MinValues, MedianValues WHERE INDUS_min = INDUS_median
    UNION ALL
    SELECT 'CHAS' AS column_name FROM MinValues, MedianValues WHERE CHAS_min = CHAS_median
    UNION ALL
    SELECT 'NOX' AS column_name FROM MinValues, MedianValues WHERE NOX_min = NOX_median
    ''',
    con,
)

,column_name
0,ZN
1,CHAS


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('column_name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

Вывод:
В данных столбцах большая часть знанений равна нулю.

4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    '''
    WITH RankedLSTAT AS (
        SELECT
            LSTAT,
            MEDV,
            RANK() OVER (ORDER BY LSTAT ASC) AS rank
        FROM boston
    )
    SELECT LSTAT, MEDV, rank
    FROM RankedLSTAT
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Вывод:
Данные показывают что более богатые районы (с низким процентом населения с более низким статусом) имеют тенденцию к более высокой стоимости недвижимости.

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
pd.read_sql(
    '''
    SELECT
        CASE
            WHEN CHAS = 1 THEN 'Граничит'
            ELSE 'Не граничит'
        END AS river_boundary,
        AVG(MEDV) AS avg_price
    FROM boston
    GROUP BY CHAS
    ''',
    con,
)

,river_boundary,avg_price
0,Не граничит,22.093843
1,Граничит,28.440000


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:

pd.read_sql(
    '''
    WITH AvgValues AS (
        SELECT
            AVG(CASE WHEN CHAS = 1 THEN CRIM END) AS CRIM_river,
            AVG(CASE WHEN CHAS = 0 THEN CRIM END) AS CRIM_no_river,
            AVG(CASE WHEN CHAS = 1 THEN ZN END) AS ZN_river,
            AVG(CASE WHEN CHAS = 0 THEN ZN END) AS ZN_no_river,
            AVG(CASE WHEN CHAS = 1 THEN INDUS END) AS INDUS_river,
            AVG(CASE WHEN CHAS = 0 THEN INDUS END) AS INDUS_no_river,
            AVG(CASE WHEN CHAS = 1 THEN NOX END) AS NOX_river,
            AVG(CASE WHEN CHAS = 0 THEN NOX END) AS NOX_no_river
        FROM boston
    )
    SELECT 'CRIM' AS column_name FROM AvgValues WHERE CRIM_river > CRIM_no_river
    UNION ALL
    SELECT 'ZN' AS column_name FROM AvgValues WHERE ZN_river > ZN_no_river
    UNION ALL
    SELECT 'INDUS' AS column_name FROM AvgValues WHERE INDUS_river > INDUS_no_river
    UNION ALL
    SELECT 'NOX' AS column_name FROM AvgValues WHERE NOX_river > NOX_no_river
    ''',
    con,
)

,column_name
0,INDUS
1,NOX


Вывод: рядом с рекой некомерческая недвижимость, которая имеет высокий уровень загрезнения (концентрация оксидов азота)

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
pd.read_sql(
    '''
    SELECT
        '10th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 10 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 10 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '20th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 20 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 20 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '30th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 30 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 30 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '40th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 40 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 40 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '50th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 50 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 50 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '60th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 60 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 60 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '70th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 70 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 70 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '80th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 80 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 80 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '90th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 90 / 100 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 90 / 100 FROM boston)) AS NOX_value
    UNION ALL
    SELECT
        '100th Percentile' AS percentile,
        (SELECT INDUS FROM boston ORDER BY INDUS LIMIT 1 OFFSET (SELECT COUNT(*) * 100 / 100 - 1 FROM boston)) AS INDUS_value,
        (SELECT NOX FROM boston ORDER BY NOX LIMIT 1 OFFSET (SELECT COUNT(*) * 100 / 100 - 1 FROM boston)) AS NOX_value
    ''',
    con,
)

,percentile,INDUS_value,NOX_value
0,10th Percentile,2.89,0.426
1,20th Percentile,4.39,0.442
2,30th Percentile,5.96,0.472
3,40th Percentile,7.38,0.507
4,50th Percentile,9.69,0.538
5,60th Percentile,12.83,0.575
6,70th Percentile,18.10,0.605
7,80th Percentile,18.10,0.668
8,90th Percentile,19.58,0.713
9,100th Percentile,27.74,0.871


Вывод: В районах с более высокой долей промышленной застройки наблюдается более высокая концентрация оксидов азота.